# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f3c9dc4346-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Running explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sports. Compare awards and participation in the competitions of the players

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q9124`    | Eliud Kipchoge| node      |
| `wd:Q853003`| athletics at the 2008 Summer Olympics – men's 200 metres |node|





Also consider

```
wd:Q9124 ?p ?obj .
```

is the BGP to retrieve all **properties of Eliud Kipchoge**


Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for runner

2. Identify the BGP for long distance running

3. Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

4. Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

5. Identify the BGP for Olympic Games

6. Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

7. For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

## Task 1
Identify the BGP for runner

In [3]:
# query example
#i find the occupation of Eliud Kipchoge
queryString = """
SELECT DISTINCT ?o ?obname
WHERE {
   # bind something
   wd:Q9124 wdt:P106 ?o.
   #get the label
   ?o sc:name ?obname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13382460'), ('obname', 'marathon runner')]
[('o', 'http://www.wikidata.org/entity/Q11513337'), ('obname', 'athletics competitor')]
[('o', 'http://www.wikidata.org/entity/Q4439155'), ('obname', 'long-distance runner')]
3


In [11]:
# query example
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q9124 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q13382460'), ('oname', 'marathon runner')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q2817771'), ('oname', "2009 World Championships in Athletics – men's 5000 metres")]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q34356762'), ('oname', "2017 World Championships in Athletics – men's 5000 metres")]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport'), ('o', 'http://www.wikidata.org/entity/Q114'), ('oname', 'Kenya')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q114'), ('oname', 'Kenya')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q1151

Final query for this task

## Task 2
Identify the BGP for long distance running

In [91]:
# query example
# as we found long-distance runner is wd:Q4439155
queryString = """
SELECT DISTINCT ?o ?oname 
WHERE {
   # bind something
   wd:Q4439155 wdt:P31 ?o.
   # get the label
   ?o sc:name ?oname.
}
limit 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession')]
1


Final query for this task

In [14]:
# query example
queryString = """
SELECT DISTINCT ?runner ?oname 
WHERE {
   # bind something
   ?runner wdt:P106 wd:Q4439155.
   # get the label
   ?runner sc:name ?oname.
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('people', 'http://www.wikidata.org/entity/Q16186777'), ('oname', 'George Malley')]
[('people', 'http://www.wikidata.org/entity/Q275831'), ('oname', 'Christelle Daunay')]
[('people', 'http://www.wikidata.org/entity/Q18132157'), ('oname', 'Carlo Martinenghi')]
[('people', 'http://www.wikidata.org/entity/Q20962093'), ('oname', 'Marielle Hall')]
[('people', 'http://www.wikidata.org/entity/Q20982761'), ('oname', 'Alina Prokopyeva')]
[('people', 'http://www.wikidata.org/entity/Q20988053'), ('oname', 'Gulzhanat Zhanatbek')]
[('people', 'http://www.wikidata.org/entity/Q9374770'), ('oname', 'Willem van der Steen')]
[('people', 'http://www.wikidata.org/entity/Q21823270'), ('oname', 'Mohamed Rutitinga')]
[('people', 'http://www.wikidata.org/entity/Q3198081'), ('oname', 'Knut Alm')]
[('people', 'http://www.wikidata.org/entity/Q23332702'), ('oname', 'Stephen Sambu')]
[('people', 'http://www.wikidata.org/entity/Q16665504'), ('oname', 'Mule Wasihun')]
[('people', 'http://www.wikidata.org/en

In [10]:

# query example
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q4439155 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8928097'), ('oname', 'Category:Long-distance runners')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q13382608'), ('oname', 'cross-country skier')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pname', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q917206'), ('oname', 'long-distance running')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12803959'), ('oname', 'runner')]
5


In [22]:
# write your final query

## Task 3
Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

In [2]:
# write your queries
# i find subclass of long-distance runner
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P279 wd:Q4439155.
   # get the label
   ?s sc:name ?sname.
   
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q13382460'), ('sname', 'marathon runner')]
[('s', 'http://www.wikidata.org/entity/Q19827218'), ('sname', 'ultramarathon runner')]
2


Final query for this task

In [ ]:
# write your final query

## Task 4
Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

In [17]:
# write your queries
# marathons is wd:Q853003
#i find the predicates of marathons
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   wd:Q853003 ?p  ?o .
   # get the label
   ?p sc:name ?pname.  
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pname', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pname', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pname', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 

Final query for this task

In [7]:
# query example
#i find wdt:P276 for location and winner wdt:P1346 in the last block


queryString = """
SELECT DISTINCT ?citizenship ?cname ?location ?winner ?wname ?pname (count(?winner) as ?win) 
WHERE {
   # bind something
   ?match  wdt:P276  ?location ;
           wdt:P1346 ?winner.
   ?winner wdt:P27 ?citizenship.
    # get the label
   ?location sc:name ?pname.
   ?winner   sc:name ?wname.
   ?citizenship sc:name ?cname.
   FILTER REGEX(?pname, 'Europe')   
}
#groub by (?citizenship)
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('citizenship', 'http://www.wikidata.org/entity/Q29999'), ('cname', 'Kingdom of the Netherlands'), ('location', 'http://www.wikidata.org/entity/Q46'), ('winner', 'http://www.wikidata.org/entity/Q6242'), ('wname', 'Marianne Vos'), ('pname', 'Europe'), ('win', '1')]
[('citizenship', 'http://www.wikidata.org/entity/Q29999'), ('cname', 'Kingdom of the Netherlands'), ('location', 'http://www.wikidata.org/entity/Q46'), ('winner', 'http://www.wikidata.org/entity/Q289826'), ('wname', 'Annemiek van Vleuten'), ('pname', 'Europe'), ('win', '1')]
[('citizenship', 'http://www.wikidata.org/entity/Q38'), ('cname', 'Italy'), ('location', 'http://www.wikidata.org/entity/Q46'), ('winner', 'http://www.wikidata.org/entity/Q26162588'), ('wname', 'Sofia Bertizzolo'), ('pname', 'Europe'), ('win', '1')]
[('citizenship', 'http://www.wikidata.org/entity/Q34'), ('cname', 'Sweden'), ('location', 'http://www.wikidata.org/entity/Q46'), ('winner', 'http://www.wikidata.org/entity/Q6033195'), ('wname', 'Gunna

In [ ]:
# write your final query

## Task 5
Identify the BGP for Olympic Games

In [97]:
# write your queries
#as in the first task we found participant in is wdt:P1344
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q9124 wdt:P1344 ?o.
   # get the label
   ?o sc:name ?oname.

}

limit 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2817771'), ('oname', "2009 World Championships in Athletics – men's 5000 metres")]
[('o', 'http://www.wikidata.org/entity/Q34356762'), ('oname', "2017 World Championships in Athletics – men's 5000 metres")]
[('o', 'http://www.wikidata.org/entity/Q18193712'), ('oname', 'athletics at the 2016 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q695233'), ('oname', '2010 Commonwealth Games')]
[('o', 'http://www.wikidata.org/entity/Q8558'), ('oname', '2004 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q1141335'), ('oname', '2006 IAAF World Indoor Championships')]
[('o', 'http://www.wikidata.org/entity/Q30687990'), ('oname', 'Breaking2')]
[('o', 'http://www.wikidata.org/entity/Q21858018'), ('oname', "athletics at the 2016 Summer Olympics – men's marathon")]
[('o', 'http://www.wikidata.org/entity/Q65242159'), ('oname', "athletics at the 2020 Summer Olympics – men's marathon")]
[('o', 'http://www.wikidata.org/entity/Q52105373')

Final query for this task

In [15]:
# write your final query
#i want to find instances of summer olyampic
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q18193712 wdt:P31 ?o.
   # get the label
   ?o sc:name ?oname.
}

limit 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q18536594'), ('oname', 'Olympic sporting event')]
[('o', 'http://www.wikidata.org/entity/Q26132862'), ('oname', 'Olympic sports discipline event')]
[('o', 'http://www.wikidata.org/entity/Q11783626'), ('oname', 'athletics meeting')]
[('o', 'http://www.wikidata.org/entity/Q715044'), ('oname', 'athletics at the Summer Olympics')]
4


In [16]:
# write your final query

queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
   # bind something
   ?s wdt:P279 wd:Q18536594.
   # get the label
   ?s sc:name ?sname.

}

limit 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q637848'), ('sname', 'Marathons at the Olympics')]
[('s', 'http://www.wikidata.org/entity/Q70106932'), ('sname', "cycling at the Summer Olympics – men's individual road race")]
[('s', 'http://www.wikidata.org/entity/Q3892502'), ('sname', 'women’s volleyball at the Summer Olympics')]
[('s', 'http://www.wikidata.org/entity/Q3892574'), ('sname', 'men’s volleyball at the Summer Olympics')]
[('s', 'http://www.wikidata.org/entity/Q105492332'), ('sname', "football at the Summer Olympics – men's tournament")]
[('s', 'http://www.wikidata.org/entity/Q2806793'), ('sname', '10,000 metres at the Olympics')]
[('s', 'http://www.wikidata.org/entity/Q105492335'), ('sname', "football at the Summer Olympics – women's tournament")]
[('s', 'http://www.wikidata.org/entity/Q2817758'), ('sname', '5000 metres at the Olympics')]
[('s', 'http://www.wikidata.org/entity/Q2814440'), ('sname', '200 metres at the Olympics')]
[('s', 'http://www.wikidata.org/entity/Q441980

In [15]:
# write your final query

queryString = """
SELECT DISTINCT ?s ?sname
WHERE {

   # bind something
   ?s wdt:P31 wd:Q18536594.
   # get the label
   ?s sc:name ?sname.

}

limit 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17088224'), ('sname', "shooting at the 1992 Summer Olympics – men's 50 metre rifle three positions")]
[('s', 'http://www.wikidata.org/entity/Q15055081'), ('sname', "snowboarding at the 2014 Winter Olympics – men's halfpipe")]
[('s', 'http://www.wikidata.org/entity/Q15506507'), ('sname', "short track speed skating at the 2014 Winter Olympics – men's 1500 metres")]
[('s', 'http://www.wikidata.org/entity/Q7500802'), ('sname', "shooting at the 1996 Summer Olympics – women's 50 metre rifle three positions")]
[('s', 'http://www.wikidata.org/entity/Q25162542'), ('sname', "rugby sevens at the 2016 Summer Olympics – men's tournament")]
[('s', 'http://www.wikidata.org/entity/Q25991469'), ('sname', "swimming at the 2016 Summer Olympics – women's 4 × 100 metre medley relay")]
[('s', 'http://www.wikidata.org/entity/Q26250761'), ('sname', "sailing at the 2016 Summer Olympics – men's RS:X")]
[('s', 'http://www.wikidata.org/entity/Q26252025'), ('sname', 

## Task 6
Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

In [17]:
# write your queries


queryString = """
SELECT DISTINCT ?match ?mname ?time 
WHERE {
   # bind something
   ?match wdt:P31 wd:Q18536594;
          wdt:P585 ?time.
   # get the label
   ?match sc:name ?mname.
   #?time sc:name ?tname.
 
}

limit 30
"""

print("Results")
x=run_query(queryString)

Results
[('match', 'http://www.wikidata.org/entity/Q17088224'), ('mname', "shooting at the 1992 Summer Olympics – men's 50 metre rifle three positions"), ('time', '1992-01-01T00:00:00Z')]
[('match', 'http://www.wikidata.org/entity/Q15055081'), ('mname', "snowboarding at the 2014 Winter Olympics – men's halfpipe"), ('time', '2014-02-11T00:00:00Z')]
[('match', 'http://www.wikidata.org/entity/Q15506507'), ('mname', "short track speed skating at the 2014 Winter Olympics – men's 1500 metres"), ('time', '2014-01-01T00:00:00Z')]
[('match', 'http://www.wikidata.org/entity/Q7500802'), ('mname', "shooting at the 1996 Summer Olympics – women's 50 metre rifle three positions"), ('time', '1996-01-01T00:00:00Z')]
[('match', 'http://www.wikidata.org/entity/Q25162542'), ('mname', "rugby sevens at the 2016 Summer Olympics – men's tournament"), ('time', '2016-01-01T00:00:00Z')]
[('match', 'http://www.wikidata.org/entity/Q25991469'), ('mname', "swimming at the 2016 Summer Olympics – women's 4 × 100 metre

Final query for this task

In [16]:
# write your queries
# i have found time as P585
#i found P1346 as winner
#2004-2008-2012-2016
queryString = """
SELECT DISTINCT ?match ?mname ?time ?rname ?runner 
WHERE {
   # bind something
   ?match  wdt:P31 wd:Q18536594;
           wdt:P585 ?time;
           wdt:P1346 ?winner.
   ?winner wdt:P106 ?runner.
   # get the label
   ?match sc:name ?mname.
   ?runner sc:name ?rname.
   FILTER REGEX(?rname, 'runner','i')
   FILTER REGEX(?mname, 'summer','i')
   FILTER(YEAR(?time) >= 2008 && YEAR(?time) <=2014)
 
}

limit 30
"""

print("Results")
x=run_query(queryString)

Results
[('match', 'http://www.wikidata.org/entity/Q2071171'), ('mname', "athletics at the 2012 Summer Olympics – men's 5000 metres"), ('time', '2012-01-01T00:00:00Z'), ('rname', 'marathon runner'), ('runner', 'http://www.wikidata.org/entity/Q13382460')]
[('match', 'http://www.wikidata.org/entity/Q2071171'), ('mname', "athletics at the 2012 Summer Olympics – men's 5000 metres"), ('time', '2012-01-01T00:00:00Z'), ('rname', 'middle-distance runner'), ('runner', 'http://www.wikidata.org/entity/Q13381753')]
[('match', 'http://www.wikidata.org/entity/Q591384'), ('mname', "athletics at the 2012 Summer Olympics – men's 1500 metres"), ('time', '2012-01-01T00:00:00Z'), ('rname', 'middle-distance runner'), ('runner', 'http://www.wikidata.org/entity/Q13381753')]
[('match', 'http://www.wikidata.org/entity/Q917328'), ('mname', "athletics at the 2008 Summer Olympics – men's 1500 metres"), ('time', '2008-01-01T00:00:00Z'), ('rname', 'middle-distance runner'), ('runner', 'http://www.wikidata.org/entit

## Task 7
For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

In [13]:
# write your queries
 
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P279 wd:Q4439155.
   # get the label
   ?s sc:name ?sname.
   
}
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q13382460'), ('sname', 'marathon runner')]
[('s', 'http://www.wikidata.org/entity/Q19827218'), ('sname', 'ultramarathon runner')]
2


Final query for this task

In [4]:
# query example
# we find record holder which is P1000 
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   wd:Q9124 ?p ?o.
   # get the label
   ?p sc:name ?pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('pname', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('pname', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('pname', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('pname', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pname', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P1741'), ('pname', 'GTAA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19')

In [4]:
# write your queries
# query example
# we should find who hold the World Record in the long distance running
queryString = """
SELECT DISTINCT ?runner ?oname ?dname ?discipline ?rname ?record
WHERE {
   # bind something
    ?discipline wdt:P279  wd:Q4439155.
    ?runner     wdt:P1000 ?record;
                wdt:P106  ?discipline.
   # get the label
   ?runner sc:name ?oname.
   ?record sc:name ?rname.
   ?discipline sc:name ?dname.
   FILTER REGEX(?rname, 'world','i')
   
}
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('runner', 'http://www.wikidata.org/entity/Q1044585'), ('oname', 'Caroline Walker'), ('dname', 'marathon runner'), ('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('rname', 'Marathon world record progression'), ('record', 'http://www.wikidata.org/entity/Q929585')]
[('runner', 'http://www.wikidata.org/entity/Q171500'), ('oname', 'Gebrselassie Haile'), ('dname', 'marathon runner'), ('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('rname', 'Marathon world record progression'), ('record', 'http://www.wikidata.org/entity/Q929585')]
[('runner', 'http://www.wikidata.org/entity/Q728634'), ('oname', 'Robert de Castella'), ('dname', 'marathon runner'), ('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('rname', 'Marathon world record progression'), ('record', 'http://www.wikidata.org/entity/Q929585')]
[('runner', 'http://www.wikidata.org/entity/Q9124'), ('oname', 'Eliud Kipchoge'), ('dname', 'marathon runner'), ('discipline', 'http://www.wikidata.org/en

In [ ]:
# write your final query